# Подсчет частотности пар товаров в продуктовых чеках

В файле содержится информация о покупках людей.

* id – означает покупку (в одну покупку входят все товары, купленные пользователем во время 1 похода в магазин)
* Товар – наименование товара
* Количество – число единиц купленного товара

Воспользуйтесь этими данными и выясните, какие пары товаров пользователи чаще всего покупают вместе. По сути, вам необходимо найти паттерны покупок, что позволит оптимизировать размещение продуктов в магазине, для удобства пользователей и увеличения выручки.

In [1]:
import pandas as pd
from itertools import combinations
import random
from tqdm.notebook import tqdm

In [2]:
list_check_number = []
list_product_name = []
list_quantity = []
for _ in range(10000):
  list_check_number.append(random.randint(1,1000))

  current_product = "pr_" + str(random.randint(1,100))
  list_product_name.append(current_product)

  list_quantity.append(random.randint(1,10))

In [3]:
d_demo_data = {'check_number': list_check_number, 'product_name': list_product_name, 'quantity': list_quantity}

In [4]:
df_sales = pd.DataFrame(data=d_demo_data)

In [5]:
df_sales = df_sales.sort_values(by=['check_number','product_name'])
df_sales.head()

,check_number,product_name,quantity
2652,1,pr_30,8
9672,1,pr_32,3
785,1,pr_60,4
5662,1,pr_88,6
854,1,pr_90,6


In [6]:
df_sales.to_csv('df_sales.csv',index=False)

In [7]:
list_unique_product_name = df_sales['product_name'].unique()

In [8]:
list_combinations_unique_product_name = list(combinations(list_unique_product_name, 2))
print(len(list_combinations_unique_product_name))
print(list_combinations_unique_product_name[:5])

4950
[('pr_30', 'pr_32'), ('pr_30', 'pr_60'), ('pr_30', 'pr_88'), ('pr_30', 'pr_90'), ('pr_30', 'pr_94')]


In [9]:
list_names_col = ['product_1','product_2']
df_product_patterns = pd.DataFrame(list_combinations_unique_product_name, columns=list_names_col)
df_product_patterns.head()

,product_1,product_2
0,pr_30,pr_32
1,pr_30,pr_60
2,pr_30,pr_88
3,pr_30,pr_90
4,pr_30,pr_94


In [10]:
df_sales_group = df_sales.groupby(by=['check_number'])['product_name'].apply(list).reset_index(name='list_product_name')
df_sales_group.head(25)

,check_number,list_product_name
0,1,"[pr_30, pr_32, pr_60, pr_88, pr_90, pr_94, pr_95]"
1,2,"[pr_18, pr_25, pr_25, pr_37, pr_45, pr_60, pr_..."
2,3,"[pr_13, pr_27, pr_31, pr_35, pr_42, pr_43, pr_..."
3,4,"[pr_41, pr_47, pr_59, pr_60, pr_7, pr_82, pr_97]"
4,5,"[pr_43, pr_45, pr_8, pr_90]"
5,6,"[pr_21, pr_27, pr_30, pr_36, pr_43, pr_58, pr_..."
6,7,"[pr_19, pr_2, pr_48, pr_56, pr_60, pr_64, pr_6..."
7,8,"[pr_1, pr_14, pr_18, pr_34, pr_41, pr_46, pr_5..."
8,9,"[pr_1, pr_4, pr_60, pr_66, pr_68, pr_87, pr_89..."
9,10,"[pr_10, pr_100, pr_20, pr_23, pr_51, pr_52, pr..."


In [11]:
list_amount = []
for product_1,product_2 in tqdm(zip(df_product_patterns['product_1'],df_product_patterns['product_2'])):
  current_amount = 0
  for list_product_name in df_sales_group['list_product_name']:
    if (product_1 in list_product_name) and \
        (product_2 in list_product_name):
         current_amount = current_amount + 1 
  list_amount.append(current_amount)

0it [00:00, ?it/s]

In [12]:
df_product_patterns['amount'] = list_amount
df_product_patterns = df_product_patterns.sort_values(by=['amount'],ascending=False)
df_product_patterns.head()

,product_1,product_2,amount
4301,pr_20,pr_49,22
4805,pr_71,pr_39,21
4618,pr_50,pr_57,20
3297,pr_19,pr_2,20
4888,pr_33,pr_57,20
